# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [5]:
year: int = 2022
topic: str = "MEDICINA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [6]:
add_list: list = [x for x in range(26, 36, 1)] + [36, 37, 38, 41, 42, 45, 47, 51, 96, 158, 168, 170]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [7]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,
2,1. Pregunta asociada a la imagen 1.
3,
4,"Varón de 45 años que, tras un cuadro de dos días"
5,"de hiporexia, astenia leve, sensación distérmi..."
6,"cefalea holocraneal, sin nauseas ni vómitos, acu-"
7,d e a u rgencias por inestabilidad en la mar...
8,Refiere otorrea fluctuan te en oído izquierdo
9,desde la infancia. Exploración neurológica: pa...


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [8]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3.0,43,1,85,1,127,4.0,169,1
1,2,3.0,44,2,86,3,128,NaN,170,2
2,3,4.0,45,1,87,4,129,4.0,171,1
3,4,1.0,46,1,88,3,130,3.0,172,3
4,5,3.0,47,4,89,4,131,2.0,173,2
5,6,2.0,48,2,90,4,132,3.0,174,1
6,7,4.0,49,4,91,1,133,4.0,175,4
7,8,1.0,50,4,92,2,134,3.0,176,4
8,9,4.0,51,4,93,1,135,2.0,177,3
9,10,2.0,52,3,94,4,136,2.0,178,4


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [9]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [10]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [11]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [12]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [13]:
raw_exam_df.head(10)

0                  1. Pregunta asociada a la imagen 1.
1     Varón de 45 años que, tras un cuadro de dos días
2    de hiporexia, astenia leve, sensación distérmi...
3    cefalea holocraneal, sin nauseas ni vómitos, acu-
4    d e  a  u rgencias por inestabilidad en la mar...
5        Refiere otorrea fluctuan te en oído izquierdo
6    desde la infancia. Exploración neurológica: pares
7        craneales normales, ma rcha con lateropulsión
8      izquierda, ligera dismetría dedo-nariz con mano
9     izquierda, Glasgow 15. En la otoscopia se obser-
Name: text, dtype: object

In [14]:
raw_exam_df.tail(10)

2975       consulta de 140/85 mmHg. Señale la medida más
2976    eficaz, de las si guientes, para evitar un nuevo
2977                              evento cardiovascular:
2978      1. Optimizar el tratamiento de la hipertensión
2979               arterial, ya que está mal controlado.
2980      2. Optimizar el tratamiento hipolipemiante, ya
2981                            que está mal controlado.
2982              3. Añadir un fármaco para la diabetes.
2983    4. Abandonar el hábito tabáquico, caminar a dia-
2984                                  rio y perder peso.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [15]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [16]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Pregunta asociada a la imagen 1. Varón de 4...
1    1. Vértigo periférico por laberintitis difusa ...
2                     2. Meningitis otógena izquierda.
3    3. Síndrome cerebeloso po r absceso intraparen...
4    4. Trombosis de seno lateral por otitis media ...
5    2. Pregunta asociada a la imagen 2. P a c i e ...
6    1. Infarto cerebral secundario a consumo de dr...
7    2. Hemorragia intraparenquimatosa profunda aso...
8    3. Hemorragia parietal lobar izquierda asociad...
9    4. Hemorragia intraparenquimatosa asociada a u...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [17]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 23
Number of rows not ending with '.', ':' or '?' = 24
[100, 121, 171, 209, 360, 374, 380, 391, 567, 698, 704, 705, 757, 791, 803, 843, 853, 887, 907, 963, 964, 975, 1006, 1042]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [18]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [19]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [21]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [100, 121, 171, 360, 374, 380, 391, 567, 698, 704, 705, 757, 791, 803, 843, 853, 887, 907, 963, 964, 975, 1006, 1042]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 100:
98                                 3. Quiste aracnoideo.
99                                        4. Metástasis.
100    21. Pregunta asociad a a la imagen 21. Mujer d...
101    17.000 leucocitos, con neutrófilos 88 %. Se mu...
102                                        1. Ooforitis.
Name: text, dtype: object
Id to fix 121:
119                         3. Espasmo esofágico difuso.
120    4. Retención alimentaria por tumor submucoso (...
121    25. Pregunta asociad a a la imagen 25. Mujer d...
122    30 años (fumó durante 10 años 1 paq/día). La a...
123         1. Atelectasia de lóbulo inferior izquierdo.
Name: text, dtype: object
Id to fix 171:
169    2. La IgA pasa de la madre al recién nacido du...
170    3. La IgA puede atravesar la barrera epitelial...
171    4. El déficit selectivo de IgA es la inmunodef...
172    1:400) y la mayoría de los casos son asintomát...
173    35. Entre las nuevas medidas de protección con...
Name: text, dtype: object
Id to fix 209:
207    

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [22]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [23]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [24]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [25]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [26]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [27]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Pregunta asociada a la imagen 1. Varón de 4...,1. Vértigo periférico por laberintitis difusa ...,2. Meningitis otógena izquierda.,3. Síndrome cerebeloso po r absceso intraparen...,4. Trombosis de seno lateral por otitis media ...
1,2. Pregunta asociada a la imagen 2. P a c i e ...,1. Infarto cerebral secundario a consumo de dr...,2. Hemorragia intraparenquimatosa profunda aso...,3. Hemorragia parietal lobar izquierda asociad...,4. Hemorragia intraparenquimatosa asociada a u...
2,3. Pregunta asociada a la imagen 3. Mujer de 7...,1. Clavo trocantérico largo acerrojado proxima...,2. Rehabilitación intensiva inmediata sin inmo...,3. Inmovilización con yeso b raquial colgante ...,4. Artroplastia glenohumeral.
3,4. Pregunta asociada a la imagen 4. Mujer de 5...,1. Síndrome coronario agudo sin elevación del ...,2. Síndrome coronario agudo sin elevación del ...,3. Síndrome coronario agudo sin elevación del ...,4. Los cambios electrocardiográficos no son di...
4,5. Pregunta asociada a la imagen 5. Varón de 5...,1. Insuficiencia tricuspídea.,2. Estenosis aórtica.,3. Miocardiopatía hipertrófica.,4. Insuficiencia mitral.
5,6. Pregunta asociada a la imagen 6. Varón de 3...,1. Para llegar a los objetivos de tratamiento ...,2. Añadir un inhibidor de la PCSK9 y solicitar...,3. Enviarle al oftalmólogo para descartar una ...,4. Añadir niacina para disminuir la lipoproteí...
6,7. Pregunta asociada a la imagen 7. Varón de 6...,1. Carcinoma quístico.,2. Enfisema pulmonar.,3. Tuberculosis cavitada.,4. Bronquiectasias.
7,8. Pregunta asociada a la imagen 8. Mujer de 8...,1. Colecistitis aguda.,2. Enfermedad de Caroli.,3. Vesícula en porcelana.,4. Cólico biliar.
8,9. Pregunta asociada a la imagen 9. En relació...,1. Su forma de debut más frecuente es un sangr...,2. La citología no es útil en el diagnóstico p...,3. Es más frecuente en mujeres jóvenes.,4. Uno de los factores de riesgo más important...
9,10. Pregunta asociad a a la imagen 10. Varón d...,1. La esplenomegalia es un hallazgo frecuente ...,2. Puede producir episodios dolorosos por cris...,3. Es debida a un defecto congénito en las pro...,4. En el examen del frotis sanguíneo se obser...


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [28]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [29]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [30]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [31]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,3.0
2,3,4.0
3,4,1.0
4,5,3.0
5,6,2.0
6,7,4.0
7,8,1.0
8,9,4.0
9,10,2.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [32]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Pregunta asociada a la imagen 1. Varón de 4...,1. Vértigo periférico por laberintitis difusa ...,2. Meningitis otógena izquierda.,3. Síndrome cerebeloso po r absceso intraparen...,4. Trombosis de seno lateral por otitis media ...,3.0
1,2. Pregunta asociada a la imagen 2. P a c i e ...,1. Infarto cerebral secundario a consumo de dr...,2. Hemorragia intraparenquimatosa profunda aso...,3. Hemorragia parietal lobar izquierda asociad...,4. Hemorragia intraparenquimatosa asociada a u...,3.0
2,3. Pregunta asociada a la imagen 3. Mujer de 7...,1. Clavo trocantérico largo acerrojado proxima...,2. Rehabilitación intensiva inmediata sin inmo...,3. Inmovilización con yeso b raquial colgante ...,4. Artroplastia glenohumeral.,4.0
3,4. Pregunta asociada a la imagen 4. Mujer de 5...,1. Síndrome coronario agudo sin elevación del ...,2. Síndrome coronario agudo sin elevación del ...,3. Síndrome coronario agudo sin elevación del ...,4. Los cambios electrocardiográficos no son di...,1.0
4,5. Pregunta asociada a la imagen 5. Varón de 5...,1. Insuficiencia tricuspídea.,2. Estenosis aórtica.,3. Miocardiopatía hipertrófica.,4. Insuficiencia mitral.,3.0
5,6. Pregunta asociada a la imagen 6. Varón de 3...,1. Para llegar a los objetivos de tratamiento ...,2. Añadir un inhibidor de la PCSK9 y solicitar...,3. Enviarle al oftalmólogo para descartar una ...,4. Añadir niacina para disminuir la lipoproteí...,2.0
6,7. Pregunta asociada a la imagen 7. Varón de 6...,1. Carcinoma quístico.,2. Enfisema pulmonar.,3. Tuberculosis cavitada.,4. Bronquiectasias.,4.0
7,8. Pregunta asociada a la imagen 8. Mujer de 8...,1. Colecistitis aguda.,2. Enfermedad de Caroli.,3. Vesícula en porcelana.,4. Cólico biliar.,1.0
8,9. Pregunta asociada a la imagen 9. En relació...,1. Su forma de debut más frecuente es un sangr...,2. La citología no es útil en el diagnóstico p...,3. Es más frecuente en mujeres jóvenes.,4. Uno de los factores de riesgo más important...,4.0
9,10. Pregunta asociad a a la imagen 10. Varón d...,1. La esplenomegalia es un hallazgo frecuente ...,2. Puede producir episodios dolorosos por cris...,3. Es debida a un defecto congénito en las pro...,4. En el examen del frotis sanguíneo se obser...,2.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [33]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [34]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [35]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [36]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Pregunta asociada a la imagen 1. Varón de 4...,1. Vértigo periférico por laberintitis difusa ...,2. Meningitis otógena izquierda.,3. Síndrome cerebeloso po r absceso intraparen...,4. Trombosis de seno lateral por otitis media ...,3,2022
1,2. Pregunta asociada a la imagen 2. P a c i e ...,1. Infarto cerebral secundario a consumo de dr...,2. Hemorragia intraparenquimatosa profunda aso...,3. Hemorragia parietal lobar izquierda asociad...,4. Hemorragia intraparenquimatosa asociada a u...,3,2022
2,3. Pregunta asociada a la imagen 3. Mujer de 7...,1. Clavo trocantérico largo acerrojado proxima...,2. Rehabilitación intensiva inmediata sin inmo...,3. Inmovilización con yeso b raquial colgante ...,4. Artroplastia glenohumeral.,4,2022
3,4. Pregunta asociada a la imagen 4. Mujer de 5...,1. Síndrome coronario agudo sin elevación del ...,2. Síndrome coronario agudo sin elevación del ...,3. Síndrome coronario agudo sin elevación del ...,4. Los cambios electrocardiográficos no son di...,1,2022
4,5. Pregunta asociada a la imagen 5. Varón de 5...,1. Insuficiencia tricuspídea.,2. Estenosis aórtica.,3. Miocardiopatía hipertrófica.,4. Insuficiencia mitral.,3,2022
5,6. Pregunta asociada a la imagen 6. Varón de 3...,1. Para llegar a los objetivos de tratamiento ...,2. Añadir un inhibidor de la PCSK9 y solicitar...,3. Enviarle al oftalmólogo para descartar una ...,4. Añadir niacina para disminuir la lipoproteí...,2,2022
6,7. Pregunta asociada a la imagen 7. Varón de 6...,1. Carcinoma quístico.,2. Enfisema pulmonar.,3. Tuberculosis cavitada.,4. Bronquiectasias.,4,2022
7,8. Pregunta asociada a la imagen 8. Mujer de 8...,1. Colecistitis aguda.,2. Enfermedad de Caroli.,3. Vesícula en porcelana.,4. Cólico biliar.,1,2022
8,9. Pregunta asociada a la imagen 9. En relació...,1. Su forma de debut más frecuente es un sangr...,2. La citología no es útil en el diagnóstico p...,3. Es más frecuente en mujeres jóvenes.,4. Uno de los factores de riesgo más important...,4,2022
9,10. Pregunta asociad a a la imagen 10. Varón d...,1. La esplenomegalia es un hallazgo frecuente ...,2. Puede producir episodios dolorosos por cris...,3. Es debida a un defecto congénito en las pro...,4. En el examen del frotis sanguíneo se obser...,2,2022


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [37]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 22
Number of rows in filtered df = 22


**2. Showing the filtered DataFrame**

Using head and tail

In [38]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,26. En un paciente con síndrome nefrótico es p...,1. Disminución de las lipoproteínas de muy baj...,2. Aumento de la apolipoproteína B-100.,3. Aumento de la apolipoproteína A-I.,4. Aumento de la actividad lipoproteínlipasa (...,2,2022
1,27. En relación con el metabolismo de compuest...,1. Sintetizar de forma limitada urea en situac...,2. Utilizar los alfa-cetoácidos derivados de l...,3. Producir alanina por transaminación a parti...,"4. Elaborar creatina utilizando glicina, ATP y...",2,2022
2,28. En la glucogenosis de t ipo Ia (enfermedad...,1. El aumento de glucógeno muscular.,2. La hipouricemia por reducción en la reabsor...,3. El aumento del lactato plasmático.,4. La disminución de la síntesis hepática de t...,3,2022
3,29. En relación con las presiones que ori gina...,1. La aspiración continua del exceso de líquid...,"2. Durante la inspiración normal, la presión p...","3. Durante la espiración normal, la presión al...",4. La presión transpulmonar (presión alveolar ...,3,2022
4,30. El flujo sanguíneo tur bulento tiende a au...,1. Viscosidad de la sangre.,2. Velocidad del flujo sanguíneo.,3. Diámetro del vaso sanguíneo.,4. Densidad de la sangre.,1,2022
5,31. En relación con la re gulación del interca...,1. La distribución del líquido entre los compa...,2. Alrededor del 80 % de la osmolaridad total ...,3. Alrededor del 50 % de la osmolaridad en el ...,4. Cambios relativamente pequeños en la concen...,3,2022
6,32. Señala la respuesta correcta respecto a la...,1. En el asa ascendente de Henle se reabsorbe ...,2. En el túbulo colector se reabsorbe un 20 % ...,3. La mayor reabsorción de sodio se produce en...,4. La reabsorción de sodio en el túbulo proxim...,1,2022
7,33. ¿Cómo se denomina el inicio pro gresivamen...,1. Penetrancia variable.,2. Anticipación genética.,3. Anticipación alélica.,4. Penetrancia completa.,2,2022
8,34. En relación con el papel de la inmuno glob...,1. La IgA activa la vía clásica del complement...,2. La IgA pasa de la madre al recién nacido du...,3. La IgA puede atravesar la barrera epitelial...,4. El déficit selectivo de IgA es la inmunodef...,1,2022
9,35. Entre las nuevas medidas de protección con...,1. Es una forma de inmunización activa cuyo re...,2. Está indicado en pacientes inmunodeprimidos...,3. Confiere protección durante un período de t...,4. Induce memoria inmunológica basada en la ac...,2,2022


In [39]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
12,38. Entre los avances en inmunoterapia antitum...,1. El elemento de reconocimiento del receptor ...,2. El receptor CAR posee un dominio intracelul...,3. Entre los efectos secundarios graves de est...,4. El reconocimiento del antígeno por el recep...,4,2022
13,41. En una enfermedad genética con un patrón d...,1. 25 % en cada embarazo independiente del sex...,2. 50 % en cada embarazo de un hijo varón.,3. 50 % en cada embarazo de una hija mujer.,4. 50 % en cada embarazo independiente del sex...,1,2022
14,42. En un determinado gen la secuencia de ADN ...,1. A la que inicia la transcripción para forma...,2. A la que no está presente en el ARN maduro.,3. A la que se transcribe para estar presente ...,4. A la que regula la expresión del gen.,2,2022
15,45. El valor predictivo negativo de una prueba...,"1. De cada 100 resultados negativos, 92 corres...","2. De cada 100 personas sanas, en 92 de ellos ...","3. De cada 100 enfermos, en 8 de ellos el resu...","4. De cada 100 resultados positivos, 8 de ello...",1,2022
16,47. ¿En cuál de las siguientes enfermedades de...,1. Sarampión.,2. Tuberculosis.,3. Norovirus.,4. Varicela.,4,2022
17,51. Las vacunas conjugadas:,1. Son vacunas de virus y bacterias combinadas...,2. Son vacunas de virus enteros conjugadas con...,3. Son vacunas bacterianas conjugadas con un p...,4. Son vacunas polisacáridas conjugadas con un...,4,2022
18,96. Respecto a la exploración neuroló gica de ...,1. Se encuentra debilidad global en todos los ...,2. El reflejo de Babinski está presente.,3. Presenta hipertonía tipo rigidez.,4. Existe hiperreflexia en los reflejos osteot...,3,2022
19,"158. En relación con la hiperprolactinemia, se...",1. La hiperprolactinemia puede estar ocasionad...,2. El hipertiroidismo provoca elevación de pro...,3. El tratamiento médico de elección son los a...,4. Causa amenorrea hipogonadotropa.,2,2022
20,168. En las faringitis causadas por Streptococ...,1. El cuadro clínico es diagnóstico: fiebre el...,2. Los test de diagnóstico rápido presentan un...,3. La pauta de tratamiento de elección es amox...,4. El tratamiento antibiótico ha demostrado re...,4,2022
21,170. Mu jer de 67 años que sufrió dos meses an...,1. Metronidazol.,2. Fidaxomicina.,3. Vancomicina.,4. Rifaximina.,2,2022


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [40]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [41]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [42]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [43]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()